# Create Dataset for training

In [1]:
from transformers import AutoTokenizer
from datasets import load_dataset
from transformers import AutoModelWithLMHead
from transformers import Trainer, TrainingArguments
import transformers
transformers.set_seed(42)

import wandb

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['WANDB_NOTEBOOK_NAME'] = 'create_dataset.ipynb'
wandb.login(key="247b3da94c9b88bd5e990f1d94799ca3ded57d6b")

/home/felix/llm/ChatTable/new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: Currently logged in as: felix-ml (fml-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/felix/.netrc


True

In [2]:
# Specify the path to your JSON file
file_path = 'berufslexikon_cleaned_manual.json'

# Load the dataset
ds = load_dataset('json', data_files=file_path)

# Specify the model checkpoint
model_checkpoint = "dbmdz/german-gpt2"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
ds["train"][1]

{'content': '3D-DesignerIn \n 3D-DesignerIn Berufsbereiche: Medien, Grafik, Design, Druck, Kunst, Kunsthandwerk Ausbildungsform: Uni/FH/PH Einstiegsgehalt lt. KV: Gehalt: € 2.210,- bis € 2.730,-  Tätigkeitsmerkmale 3D-DesignerInnen entwerfen, erstellen und programmieren komplexe, dreidimensionale digitale Objekte. Da sind z.B. Figuren für Videospiele und Filme, Grafiken, bewegte Bilder oder räumlich wirkende Darstellungen. Sie animieren auch Standbilder aus 3D-Scans. Außerdem programmieren sie interaktive 3D-Echtzeitgrafiken, zum Beispiel ein dreidimensionales Modell einer realen Szene (Virtuelle Realität).Digitale Objekte dienen zur Visualisierung in allen möglichen Bereichen wie Architektur, Raumplanung, Verkehr und Sport oder für die Visualisierung in technischen Bereichen. Die DesignerInnen nutzen 3D-Modeling-Software (z.B. AutoCad) und Simulationssoftware für interaktive 3D-Echtzeitgrafiken. Besondere Bedeutung hat die Visualisierung von Objekten auch für das E-Learning sowie für 

In [4]:
# count the number of words in the dataset
total_words = 0
for example in ds["train"]:
    total_words += len(example["profession"].split())
    total_words += len(example["content"].split())
print(f"Total number of words in the dataset: {total_words}")


Total number of words in the dataset: 3473496


In [5]:
# As this dataset has no validation split, we will create one
ds = ds["train"].train_test_split(test_size=0.2, seed=42)

In [6]:
model = AutoModelWithLMHead.from_pretrained(model_checkpoint)

/home/felix/llm/ChatTable/new_env/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1564: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [7]:
type(ds["train"][1]["profession"])

str

In [8]:
# We'll create a tokenizer from model checkpoint
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)#, use_fast=False)

# We'll need padding to have same length sequences in a batch
tokenizer.pad_token = tokenizer.eos_token

# Define a tokenization function that first concatenates text and target
def tokenize_function(example):
    # concatenate all texts and tokenize as one sample
    # merged = str(example["profession"]) + " " + str(example["url"]) + " " + str(example["content"])
    # print(type(merged))
    # print(merged)
    merged =  example["content"]
    batch = tokenizer(merged, padding='max_length', truncation=True, max_length=128)
    batch["labels"] = batch["input_ids"].copy()
    return batch

# def tokenize_function(example):
#     # If the fields are lists of strings, join the strings. Otherwise, use the fields as they are.
#     profession = ' '.join(example["profession"]) if isinstance(example["profession"], list) else example["profession"]
#     url = ' '.join(example["url"]) if isinstance(example["url"], list) else example["url"]
#     content = ' '.join(example["content"]) if isinstance(example["content"], list) else example["content"]

#     merged = profession + " " + url + " " + content
#     batch = tokenizer(merged, padding='max_length', truncation=True, max_length=2048)
#     batch["labels"] = batch["input_ids"].copy()
#     return batch

# Apply it on our dataset, and remove the text columns
tokenized_datasets = ds.map(tokenize_function, batched=True, remove_columns=["profession", "url", "content"])

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1122
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 281
    })
})

In [10]:
# Let's check out one prepared example
print(tokenizer.decode(tokenized_datasets["train"][ 1]["input_ids"]))

EisenbahnbetriebsleiterIn 
 EisenbahnbetriebsleiterIn Berufsbereiche: Büro, Marketing, Finanz, Recht, Sicherheit / Handel, Logistik, Verkehr Ausbildungsform: Kurz-/Spezialausbildung Einstiegsgehalt: Gehalt: nicht bekannt Beruf merken als PDF anzeigen Hinweis Die betriebsinterne Ausbildung zu diesem Beruf kann nur im Rahmen eines aufrechten Dienstverhältnisses absolviert werden. Tätigkeitsmerkmale EisenbahnbetriebsleiterInnen leiten und überwachen die sicherheitsrelevanten Prozesse in einem Eisenbahnunternehmen und tragen die Verantwortung für einen sicheren und rechtmäßigen Eisenbahnbetrieb. Sie sind dafür verantwortlich, dass die Züge, Bahnanlagen und Bauwerke einwandfrei funktionieren. Im Rahmen ihrer Tätigkeit kennen EisenbahnbetriebsleiterInnen die allgemein


# Training

In [11]:
# Start a new wandb run
run = wandb.init(project="german-gpt2-ams-finetune", job_type="train")

In [12]:
import torch

# If CUDA is available, use it
if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cudnn.benchmark = True
    print(f"Using {torch.cuda.device_count()} GPUs.")
else:
    device = torch.device('cpu')
    print("No GPU available, using the CPU instead.")

Using 1 GPUs.


In [13]:
# Define the training arguments
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-ams-finetuned",
    report_to="wandb", # we need one line to track experiments in wandb
    num_train_epochs=2,
    logging_steps=1,
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    weight_decay=0.01,
    #no_cuda=False, # force cpu use, will be renamed `use_cpu`
)

In [14]:
# We'll use HF Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [15]:
# Let's train!
trainer.train()

Epoch,Training Loss,Validation Loss
1,1.505300,1.332106
2,1.143700,1.304300


TrainOutput(global_step=282, training_loss=1.3542108461789206, metrics={'train_runtime': 77.2676, 'train_samples_per_second': 29.042, 'train_steps_per_second': 3.65, 'total_flos': 146584829952000.0, 'train_loss': 1.3542108461789206, 'epoch': 2.0})

In [16]:
#save the model
trainer.save_model(f"{model_name}-ams-finetuned")

In [17]:
transformers.logging.set_verbosity_error() # suppress tokenizer warnings

prefix = "Hilf bei der Suche nach einem Beruf:"

prompts = [
    "Ich bin 15 Jahre alt, gut im Umgang mit Menschen und möchte gerne einen Beruf erlernen, in dem ich viel mit Menschen zu tun habe.",
    "Ich bin 30 Jahre alt und mag Tier sehr gerne. Ich möchte gerne einen Beruf erlernen, in dem ich viel mit Tieren zu tun habe.",
    "Ich bin 19 und möchte etwas technisches machen. Ich bin sehr gut in Mathe und Physik.",
]

table = wandb.Table(columns=["prompt", "generated", "input_tokens", "generated_tokens"])

for prompt in prompts:
    # encode the prompt and generate text until the output length (which includes the context length) reaches 50
    input_ids = tokenizer.encode(prefix + prompt, return_tensors='pt').to(device)
    output = model.generate(input_ids, max_length=128, num_beams=5, early_stopping=True)
    generated = tokenizer.decode(output[0], skip_special_tokens=True)
    table.add_data(prompt, generated, len(input_ids[0]), len(output[0]))
    
wandb.log({"generated": table})

In [ ]:
wandb.finish()